<center><img src="https://drive.google.com/uc?id=1Z3JvAFmL2IkBnQmmt5f4uTcXVhO5f7cq"/></center>

------
<center>&copy; Research Group CAMMA, University of Strasbourg, <a href="http://camma.u-strasbg.fr">http://camma.u-strasbg.fr</a>

<h2>Authors: Vinkle Srivastav, Idris Hamoud, Ege Oszoy </h2>
</center>

------

# <center><font color=green> Lecture: Human Pose Estimation in the Operating Room </font></center>
<center><img src="https://lh3.googleusercontent.com/drive-viewer/AFGJ81qU1PeDDhk9Tl0tfMwk9pMOGEz__WBAXMU6Fo4hZXgz20jwymqYgGL4PS6J0BxihQIeRmsv3nR14cgszU2YLKcYV6AefA=s1600"/></center>


### **Objectives**:
  1. Understand how Pose estimation can be used to understand body language and non-verbal communication cues. It can be used in healthcare for patient monitoring or in this case in the Operating Room for workflow monitoring.
  2. PyTorch `Dataset` and `Dataloader` for subset of MVOR dataset
  3. Run off-the-shelf top-down approaches for HPE using Cascade Mask R-CNN as an object detector, and compare with groundtruth bounding boxes

## Setup

In [ ]:
#@title Install external dependencies
# install dependencies
# !pip install numpy
# !pip install matplotlib
# !pip install torch
# !pip install torchvision
# !pip install tqdm
# !pip install ipywidgets
# !pip install -U openmim
# !mim install mmengine
# !mim install "mmcv>=2.0.0"
# !mim install "mmdet>=3.0.0"


In [1]:
#@title Download resources
ROOT_DIR = "./HPE_dataset"

In [2]:
#@title Imports
import os
#import sys
import json
import torch
import torch.nn as nn
#import torch.nn.functional as F
from torch import optim
#import torchvision
import numpy as np
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.dpi'] = 100
#from PIL import Image
import cv2
#import pickle
import random
import ipywidgets as wd
#import glob
#import io
import mmcv
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from mmcv import Config
from mmpose.apis import (inference_top_down_pose_model, inference_bottom_up_pose_model,
                         vis_pose_result, process_mmdet_results, init_pose_model)
from mmdet.apis import inference_detector, init_detector
from pycocotools.coco import COCO
#from mmpose.datasets import DatasetInfo

torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

/Users/egeozsoy/venvs/EDU4SDS23/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:28: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


ImportError: dlopen(/Users/egeozsoy/venvs/EDU4SDS23/lib/python3.7/site-packages/mmcv/_ext.cpython-37m-darwin.so, 0x0002): Symbol not found: __Z16THPVariable_WrapN2at6TensorE
  Referenced from: <361BFD06-3F0D-3300-88B3-9C796D1F5279> /Users/egeozsoy/venvs/EDU4SDS23/lib/python3.7/site-packages/mmcv/_ext.cpython-37m-darwin.so
  Expected in:     <AA69D6A3-7D5A-3F97-B7B9-F6F15CFF01B4> /Users/egeozsoy/venvs/EDU4SDS23/lib/python3.7/site-packages/torch/lib/libtorch_python.dylib

## Data Exploration
### HPE helpers
We will in a first step visualize the keypoint annotations on one sample image from the MVOR dataset. This example will allow us to better understand the different helper functions from the mmpose library and get a better understanding of the python visualization libraries like matplotlib.

In [ ]:
# Define the day and the name of the image we use for visualization
sample_frame_info = ("day1", "000013.png")
_,axs = plt.subplots(1,3,dpi=300)
# We will plot the multiview images from the three different cameras
for cam_id in range(3):
    rgb_img_path = os.path.join(ROOT_DIR, "camma_mvor_dataset", sample_frame_info[0],
                            "cam"+str(cam_id+1),"color", sample_frame_info[1])
    rgb_img = mmcv.imread(rgb_img_path, channel_order='rgb')
    axs[cam_id].imshow(rgb_img); axs[cam_id].set_title("Camera "+str(cam_id+1))
    axs[cam_id].axis('off')

In [ ]:
# Helper function for visualization of depth maps
def normalize_depth_for_display(depth, pc=60, crop_percent=0., normalizer=None, cmap='viridis'):
    # convert to disparity
    depth = 1./(depth + 1e-6)
    if normalizer is not None:
        depth = depth/normalizer
    else:
        depth = depth/(np.percentile(depth, pc) + 1e-6)
    depth = np.clip(depth, 0, 1)
    depth = rgb2gray(depth)
    keep_H = int(depth.shape[0] * (1-crop_percent))
    depth = depth[:keep_H]

    depth = depth
    return depth 

In [ ]:
# Define the day and the name of the image we use for visualization
sample_frame_info = ("day1", "000013.png")
_,axs = plt.subplots(1,3,dpi=300)
# We will plot the multiview images from the three different cameras
for cam_id in range(3):
    depth_img_path = os.path.join(ROOT_DIR, "camma_mvor_dataset", sample_frame_info[0],
                            "cam"+str(cam_id+1),"depth", sample_frame_info[1])
    depth_img = mmcv.imread(depth_img_path, channel_order='rgb')
    depth_img = normalize_depth_for_display(depth_img)
    axs[cam_id].imshow(depth_img,'jet'); axs[cam_id].set_title("Camera "+str(cam_id+1))
    axs[cam_id].axis('off')

In [ ]:
# Load annotations from json file
annot_path = os.path.join(ROOT_DIR, "camma_mvor_2018.json")

## Top-down approaches
### Using off-the-shelf human detector


In [ ]:
# Initialization of the human detector using mmdet api
det_config = 'configs/cascade_rcnn_x101_64x4d_fpn_coco.py'
pose_config = 'configs/hrnet_w48_coco_384x288.py'
det_model = init_detector(det_config, checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_1x_coco/cascade_mask_rcnn_x101_64x4d_fpn_1x_coco_20200203-9a2db89d.pth',device='cpu') 
pose_model = init_pose_model(pose_config, checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_384x288-314c8528_20200708.pth',device='cpu')

In [ ]:
rgb_img_path =  os.path.join(ROOT_DIR, "camma_mvor_dataset", sample_frame_info[0],
                          "cam"+str(1),"color", sample_frame_info[1]) #'/raid/coco/images/test2017/000000000725.jpg'
rgb_img = mmcv.imread(rgb_img_path, channel_order='rgb')

In [ ]:
# test a single image, the resulting box is (x1, y1, x2, y2)
mmdet_results = inference_detector(det_model, rgb_img)
# keep the person class bounding boxes and process the results in an np.array format.
person_results = process_mmdet_results(mmdet_results, cat_id=1)
#person_results = [{'bbox':np.array(box['bbox']), 'device':'cpu'} for box in person_results]
#print(person_results)

In [ ]:
pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        rgb_img,
        person_results,
        bbox_thr=0.,
        format='xyxy',
        return_heatmap=True)

In [ ]:
vis_result = vis_pose_result(
        pose_model,
        rgb_img,
        pose_results,
        out_file=os.path.join(ROOT_DIR,"test_pose.png"))

plt.imshow(vis_result)

In [ ]:
_,axs = plt.subplots(1,3,dpi=300)
for cam_id in range(3):
    rgb_img_path = os.path.join(ROOT_DIR, "camma_mvor_dataset", sample_frame_info[0],
                            "cam"+str(cam_id+1),"color", sample_frame_info[1])
    rgb_img = mmcv.imread(rgb_img_path, channel_order='rgb')
    # test a single image, the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, rgb_img)
    # keep the person class bounding boxes and process the results in an np.array format.
    person_results = process_mmdet_results(mmdet_results, 1)
    #person_results = [{'bbox':np.array(box['bbox'].cpu()), 'device':'cpu'} for box in person_results]
    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        rgb_img_path,
        person_results,
        bbox_thr=0.,
        format='xyxy',
        return_heatmap=True)
    vis_result = vis_pose_result(
            pose_model,
            rgb_img,
            pose_results)
    axs[cam_id].imshow(vis_result); axs[cam_id].set_title("Camera "+str(cam_id+1))
    axs[cam_id].axis('off')

In [ ]:
with open(annot_path) as f:
   data = json.load(f)

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(annot_path)

### Using groundtruth bounding boxes

In [ ]:
# Helper function to find ids for specific filename
def find_id(annots_gt, file_name):
    for img in annots_gt.imgs.values():
        if img['file_name']==file_name:
            return img['id']
        

In [ ]:
# Load annotations for sample image defined above
id_img = find_id(coco, sample_frame_info[0] + "/cam"+str(cam_id+1)+"/color/"+ sample_frame_info[1])
anns = coco.getAnnIds(id_img)
person_results = [{'bbox':ann['bbox']+[1.0]} for ann in coco.loadAnns(anns)]

In [ ]:
pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        rgb_img,
        person_results,
        bbox_thr=0.1,
        format='xywh',
        return_heatmap=True)

In [ ]:
vis_result = vis_pose_result(
        pose_model,
        rgb_img,
        pose_results,
        out_file=os.path.join(ROOT_DIR,"test_pose.png"))

plt.imshow(vis_result)

In [ ]:
returned_outputs[0]['heatmap'].shape
rgb_img.shape
returned_outputs[0].keys()


In [ ]:
PARTS = {
    0: 'NOSE',
    1: 'LEFT_EYE',
    2: 'RIGHT_EYE',
    3: 'LEFT_EAR',
    4: 'RIGHT_EAR',
    5: 'LEFT_SHOULDER',
    6: 'RIGHT_SHOULDER',
    7: 'LEFT_ELBOW',
    8: 'RIGHT_ELBOW',
    9: 'LEFT_WRIST',
    10: 'RIGHT_WRIST',
    11: 'LEFT_HIP',
    12: 'RIGHT_HIP',
    13: 'LEFT_KNEE',
    14: 'RIGHT_KNEE',
    15: 'LEFT_ANKLE',
    16: 'RIGHT_ANKLE'
}

for joint in PARTS:
    heatmap = returned_outputs[0]['heatmap'][0,joint]
    scores = 1/(1 + np.exp(-heatmap))
    x,y = np.unravel_index(np.argmax(scores[:,:]), scores[:,:].shape)
    print(x,y)
    

## Finetune model on MVOR ?

In [ ]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
valid_dataset = eval('dataset.'+cfg.DATASET.DATASET)(
    cfg, cfg.DATASET.ROOT, cfg.DATASET.TEST_SET, False,
transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)
# valid_loader = torch.utils.data.DataLoader(
#     valid_dataset,
#     batch_size=cfg.TEST.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
#     shuffle=False,
#     num_workers=cfg.WORKERS,
#     pin_memory=True
# )